## Julia Gajtkowska lab 3

1. Znajdź liczbę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [1]:
import numpy as np
import pickle

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

from typing import Union, List, Tuple

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');


In [2]:
# Zadanie 1 

q = f'''select film.length, count(film.title) from film
        where film.length = 48 group by film.length
        '''
print(pd.read_sql(q,con=connection))


   length  count
0      48     11


In [3]:
# Zadanie 2

q = f"""select customer.first_name, customer.last_name, city.city from customer
        join address on customer.address_id = address.address_id
        join city on address.city_id = city.city_id
        where city.city = 'Adana'
        """
print(pd.read_sql(q,con=connection))


  first_name last_name   city
0      Larry  Thrasher  Adana


In [4]:
# Zadanie 3

q = f"""select film.length, avg(payment.amount) from film
        join inventory on inventory.film_id=film.film_id
        join rental on rental.inventory_id = inventory.inventory_id
        join payment on payment.rental_id = rental.rental_id
        where film.title = 'Youth Kick' group by film.length
        """
# q = f"""select * from film"""
print(pd.read_sql(q,con=connection))


   length       avg
0     179  2.823333


In [5]:
# Zadanie 4

q = f"""select category.name, count(film.title) from category
        join film_category on category.category_id = film_category.category_id
        join film on film_category.film_id = film.film_id
        group by category.name
        """

print(pd.read_sql(q,con=connection))


           name  count
0        Horror     56
1        Comedy     58
2        Sci-Fi     61
3         Drama     62
4       Foreign     73
5      Classics     57
6         Games     61
7           New     63
8        Travel     57
9         Music     51
10       Action     64
11  Documentary     68
12       Sports     74
13    Animation     66
14       Family     69
15     Children     60


In [6]:
# Zadanie 5

q = f"""select country.country, count(customer.first_name) from country
        join city on country.country_id = city.country_id
        join address on city.city_id = address.city_id
        join customer on address.address_id = customer.address_id
        group by country.country
        """

print(pd.read_sql(q,con=connection))


                  country  count
0                Cambodia      2
1                  Turkey     15
2                 Germany      7
3              Madagascar      1
4                    Chad      1
..                    ...    ...
103                 Sudan      2
104  United Arab Emirates      3
105              Tanzania      3
106      French Polynesia      2
107              Bulgaria      2

[108 rows x 2 columns]


In [7]:
# Zadanie 6 

q = f"""select store.manager_staff_id, store.address_id from store
        join staff on store.store_id = staff.store_id
        join rental on staff.staff_id = rental.staff_id
        join customer on rental.customer_id = customer.customer_id
        group by store.manager_staff_id, store.address_id
        having count(distinct (customer.first_name || ' '|| customer.last_name)) > 100 
        and count(distinct (customer.first_name || ' '|| customer.last_name)) <300
        """

print(pd.read_sql(q,con=connection))


Empty DataFrame
Columns: [manager_staff_id, address_id]
Index: []


In [8]:
# Zadanie 7

q = f"""select customer.first_name, customer.last_name, sum(film.length) from customer
        join rental on customer.customer_id = rental.customer_id
        join inventory on rental.inventory_id = inventory.inventory_id
        join film on inventory.film_id = film.film_id
        group by customer.first_name, customer.last_name
        having sum(film.length) > 200
        """

print(pd.read_sql(q,con=connection))


    first_name    last_name   sum
0        Scott      Shelley  2916
1          Tim         Cary  4476
2       Samuel       Marlow  2291
3         Vera        Mccoy  2275
4       Nelson  Christenson  2119
..         ...          ...   ...
594      Erica     Matthews  2727
595    Stanley    Scroggins  3694
596      Ellen      Simpson  2977
597      Patsy     Davidson  3513
598  Gwendolyn          May  2890

[599 rows x 3 columns]


In [21]:
# Zadanie 8 

q = f"""select avg(payment.amount) as payment_average, film.title from payment
        inner join rental on payment.rental_id = rental.rental_id
        inner join inventory on rental.inventory_id = inventory.inventory_id
        inner join film on inventory.film_id = film.film_id
        group by film.film_id"""


print(pd.read_sql(q,con=connection))


     payment_average                  title
0           5.775714    Dragonfly Strangers
1           4.704286  Bulworth Commandments
2           3.836154       English Bulworth
3           2.545000        Pirates Roxanne
4           8.365000      Caribbean Liberty
..               ...                    ...
953         5.531667           Durham Panky
954         4.519412         National Story
955         4.528462          Nash Chocolat
956         5.523333        Airplane Sierra
957         5.490000         Image Princess

[958 rows x 2 columns]


In [22]:
# Zadanie 9

q = f"""select category.name, avg(film.length) from category
        join film_category on category.category_id = film_category.category_id
        join film on film_category.film_id = film.film_id
        group by category.name
        """

print(pd.read_sql(q,con=connection))


           name         avg
0        Horror  112.482143
1        Comedy  115.827586
2        Sci-Fi  108.196721
3         Drama  120.838710
4       Foreign  121.698630
5      Classics  111.666667
6         Games  127.836066
7           New  111.126984
8        Travel  113.315789
9         Music  113.647059
10       Action  111.609375
11  Documentary  108.750000
12       Sports  128.202703
13    Animation  111.015152
14       Family  114.782609
15     Children  109.800000


In [23]:
# Zadanie 10


q = f"""select category.name, max(length(film.title)) from film 
        join film_category on film.film_id = film_category.film_id
        join category on film_category.category_id = category.category_id
        group by category.name        
        """

print(pd.read_sql(q,con=connection))


           name  max
0        Horror   27
1        Comedy   23
2        Sci-Fi   20
3         Drama   22
4       Foreign   20
5      Classics   23
6         Games   21
7           New   21
8        Travel   23
9         Music   22
10       Action   23
11  Documentary   22
12       Sports   25
13    Animation   22
14       Family   22
15     Children   20


In [24]:
# Zadanie 11

q = f"""select max(film.length), category.name from film
        join film_category on film.film_id = film_category.film_id
        join category on film_category.category_id = category.category_id
        group by category.name
        """

print(pd.read_sql(q,con=connection))


    max         name
0   181       Horror
1   185       Comedy
2   185       Sci-Fi
3   181        Drama
4   184      Foreign
5   184     Classics
6   185        Games
7   183          New
8   185       Travel
9   185        Music
10  185       Action
11  183  Documentary
12  184       Sports
13  185    Animation
14  184       Family
15  178     Children
